In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
tf.set_random_seed(777)

In [4]:
# parameters
log_dir = './logs/ML_Lab_11-07-Deep CNN'
learning_rate = 0.001
batch_size = 100
num_epochs = 50

In [5]:
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [6]:
X = tf.placeholder(shape=[None, 784], dtype=tf.float32, name='X')

In [7]:
X_img = tf.reshape(X, shape=[-1, 28, 28, 1])

In [8]:
Y = tf.placeholder(shape=[None,  10], dtype=tf.float32, name='Y')

In [9]:
dropout_rate = tf.placeholder(shape=None, dtype=tf.float32)

In [10]:
with tf.variable_scope('Conv1') as scope:
    W1 = tf.Variable(tf.random_normal(shape=[3, 3, 1, 32], stddev=0.01), dtype=tf.float32, name='weight1')
    L1 = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding='SAME')
    L1 = tf.nn.relu(L1)
    L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    L1 = tf.nn.dropout(L1, rate=dropout_rate)
    print(L1)

Tensor("Conv1/dropout/mul_1:0", shape=(?, 14, 14, 32), dtype=float32)


In [11]:
with tf.variable_scope('Conv2') as scope:
    W2 = tf.Variable(tf.random_normal(shape=[3, 3, 32, 64], stddev=0.01), dtype=tf.float32, name='weight2')
    L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
    L2 = tf.nn.relu(L2)
    L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    L2 = tf.nn.dropout(L2, rate=dropout_rate)
    print(L2)

Tensor("Conv2/dropout/mul_1:0", shape=(?, 7, 7, 64), dtype=float32)


In [12]:
with tf.variable_scope('Conv3') as scope:
    W3 = tf.Variable(tf.random_normal(shape=[3, 3, 64, 128], stddev=0.01), dtype=tf.float32, name='weight3')
    L3 = tf.nn.conv2d(L2, W3, strides=[1, 1, 1, 1], padding='SAME')
    L3 = tf.nn.relu(L3)
    L3 = tf.nn.max_pool(L3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    L3 = tf.nn.dropout(L3, rate=dropout_rate)
    print(L3)

Tensor("Conv3/dropout/mul_1:0", shape=(?, 4, 4, 128), dtype=float32)


In [13]:
with tf.variable_scope('Fuuly-connected-1') as scope:
    L3 = tf.reshape(L3, shape=[-1, 4 * 4 * 128])
    W4 = tf.get_variable('weight4', shape=[4 * 4 * 128, 625], initializer=tf.contrib.layers.xavier_initializer())
    b4 = tf.Variable(tf.random_normal(shape=[625]), dtype=tf.float32, name='bias4')
    L4 = tf.matmul(L3, W4) + b4
    L4 = tf.nn.relu(L4)
    L4 = tf.nn.dropout(L4, rate=dropout_rate)
    print(L4)

Tensor("Fuuly-connected-1/dropout/mul_1:0", shape=(?, 625), dtype=float32)


In [14]:
with tf.variable_scope('Fuuly-connected-2') as scope:
    W5 = tf.get_variable('weight5', shape=[625, 10], initializer=tf.contrib.layers.xavier_initializer())
    b5 = tf.Variable(tf.random_normal(shape=[10]), dtype=tf.float32, name='bias5')
    hypothesis = tf.matmul(L4, W5) + b5
    print(hypothesis)

Tensor("Fuuly-connected-2/add:0", shape=(?, 10), dtype=float32)


In [15]:
with tf.variable_scope('cost'):
    cost_i = tf.nn.softmax_cross_entropy_with_logits_v2(logits=hypothesis, labels=Y)
    cost = tf.reduce_mean(cost_i)
    tf.summary.scalar('cost', cost)

In [16]:
with tf.variable_scope('Optimizer'):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [17]:
with tf.variable_scope('accuracy') as scope:
    is_correct = tf.equal(tf.argmax(hypothesis, 1), tf.arg_max(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
    tf.summary.scalar('accuracy', accuracy)

Instructions for updating:
Use `tf.math.argmax` instead


In [18]:
with tf.Session() as sess:
    # tensorboard --logdir=./logs/Deep_NN
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter(log_dir)
    writer.add_graph(sess.graph)
    
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())
    # Training cycle
    print('Learning stared. It takes sometime.')
    for epoch in range(num_epochs):
        avg_cost = 0
        total_batch = int(mnist.train.num_examples / batch_size)
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            c_val, _ = sess.run([cost, optimizer], feed_dict={X: batch_xs, Y: batch_ys, dropout_rate: 0.3})
            avg_cost += c_val / total_batch
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))
        print('Accuracy: ', sess.run(accuracy, feed_dict={X:mnist.test.images, Y: mnist.test.labels, dropout_rate: 0.}))
        summary = sess.run(merged_summary, feed_dict={X:mnist.test.images, Y: mnist.test.labels, dropout_rate: 0.})
        writer.add_summary(summary, global_step=(epoch))
    print('Learning Finished!')

Learning stared. It takes sometime.
Epoch: 0001 cost = 0.377796109
Accuracy:  0.979
Epoch: 0002 cost = 0.099399639
Accuracy:  0.9858
Epoch: 0003 cost = 0.075085009
Accuracy:  0.9866
Epoch: 0004 cost = 0.061145273
Accuracy:  0.9885
Epoch: 0005 cost = 0.051878434
Accuracy:  0.9903
Epoch: 0006 cost = 0.046417978
Accuracy:  0.9914
Epoch: 0007 cost = 0.042170126
Accuracy:  0.9903
Epoch: 0008 cost = 0.041019283
Accuracy:  0.9922
Epoch: 0009 cost = 0.036065430
Accuracy:  0.991
Epoch: 0010 cost = 0.034491181
Accuracy:  0.9926
Epoch: 0011 cost = 0.030383588
Accuracy:  0.9933
Epoch: 0012 cost = 0.030188141
Accuracy:  0.9926
Epoch: 0013 cost = 0.026632287
Accuracy:  0.9938
Epoch: 0014 cost = 0.028274718
Accuracy:  0.9915
Epoch: 0015 cost = 0.026466253
Accuracy:  0.9937
Epoch: 0016 cost = 0.023446215
Accuracy:  0.9932
Epoch: 0017 cost = 0.022636689
Accuracy:  0.9903
Epoch: 0018 cost = 0.023605787
Accuracy:  0.9933
Epoch: 0019 cost = 0.024080276
Accuracy:  0.9934
Epoch: 0020 cost = 0.021933389
Accu